# Benchmarking

## Introduction

**BTB** provides a benchmarking framework that allows users and developers to evaluate the
performance of the BTB Tuners for Machine Learning Hyperparameter Tuning on hundreds of real world
classification problem and classical mathematical optimization problems.

### Prerequisites

In order to use this `benchmarking` module, you will have to fork the
**BTB** repository
and install it from source. You can visit our
[Get Started](https://hdi-project.github.io/BTB/contributing.html#get-started)
tutorial and follow until step 4, which explains how to clone and install the repository
from it's source.

### The Benchmarking process

The Benchmarking BTB process has two main concepts.

#### Challenges

A Challenge of the BTB Benchmarking framework is a Python class which has a method that produces a
score that can be optimized by tuning a set of hyperparameters.

#### Tuning Functions

In the context of the BTB Benchmarking, `Tuning Functions` are python functions that, given a scoring
function and its tunable hyperparameters, try to search for the ideal hyperparameter values within
a given number of iterations.

If you want to add a tuner, you could follow the specific signature a tuning function has:

```python3
def tuning_function(
    scoring_function: callable,
    tunable_hyperparameters: dict,
    iterations: int) -> score: float
```


### Running the Benchmarking

The user API for the BTB Benchmarking is the `btb_benchmark.main.run_benchmark` function.

The `run_benchmark` function has the following arguments:

- `tuners`: list of tuners that will be benchmarked.
- `challenge_types`: list of types of challenges that will be used for benchmark (optional).
- `challenges`: list of names of challenges that will be benchmarked (optional).
- `sample`: if specified, run the benchmark on a subset of the available challenges of the given size (optional).
- `iterations`: the number of tuning iterations to perform per challenge and tuner.
- `output_path`: If given, store the benchmark results in the given path as a CSV file.

## Example

In this example we will create a `tuning_function` for a `GPEiTuner`,
then we will import some `challenges` and we will run a benchmark
over our `GPEiTuner` against a `Uniform` tuner already implemented
in `btb.benchmark`.

To start, we will import our `GPEiTuner` and we will create the
tuning function that instantiates a `Tunable` that will be used
by our `tuner`.

In [1]:
import numpy as np

from btb.tuning import GPEiTuner, Tunable

def tuning_function(scoring_function, tunable_hp, iterations):
    tunable = Tunable.from_dict(tunable_hp)
    tuner = GPEiTuner(tunable)
    
    best_score = -np.inf
    
    for _ in range(iterations):
        proposal = tuner.propose()
        score = scoring_function(**proposal)
        tuner.record(proposal, score)
        
        best_score = max(score, best_score)
        
    return best_score

Now we can run our `tuning_function` against a set of mathematical
optimization problems and see how it performs by giving as arguments
`tuners=tuning_function` and `challenge_types=math`:

In [2]:
from btb_benchmark import run_benchmark

scores = run_benchmark(
    tuners=tuning_function, challenge_types='math', iterations=10)

In [3]:
scores

,tuning_function
Bohachevsky(),-489.600000
Branin(),-7.961228
Rosenbrock(),-16901.000000


Now, let's score our tuning function against an already implemented
`BTB.Uniform` tuning function, by creating a list of the tuners we 
want to use:

In [4]:
tuners = [tuning_function, 'BTB.UniformTuner']

scores = run_benchmark(
    tuners=tuners,
    challenge_types='math',
    iterations=50
)
scores

,BTB.UniformTuner,tuning_function
Bohachevsky(),-99.600000,-1.600000
Branin(),-0.674776,-0.402413
Rosenbrock(),-6404.000000,-436.000000


We can also specify the challenges that we would like to run,
so let's just run our benchmarking against the `Rosenbrock` function:

In [5]:
scores = run_benchmark(
    tuners=tuners,
    challenges='rosenbrock',
    iterations=100
)
scores

,BTB.UniformTuner,tuning_function
Rosenbrock(),-400,-64


We can also instantiate a `challenge` and use it, like we did with
our `tuning_function`. Let's import a Machine Learning challenge and
instantiate it:

In [6]:
from btb_benchmark.challenges import SGDChallenge

challenge = SGDChallenge('pollution_1')
scores = run_benchmark(
    tuners=tuners,
    challenges=challenge,
    iterations=10
)

In [7]:
scores

,BTB.UniformTuner,tuning_function
SGDChallenge('pollution_1'),0.646401,0.605132


If you have created a `tuner` based on `btb.tuning.tuners.base.BaseTuner` you can
directly pass it as a `tuenrs` argument and it will create a `TuningFunction` similar
to the one created previously.

In [8]:
run_benchmark(tuners=GPEiTuner, challenges=challenge, iterations=10)

,GPEiTuner
SGDChallenge('pollution_1'),0.643693
